# Benchmark of Python OR-Tools Distance Matrix ingestion approaches


`RegisterTransitCallback` takes a long time to ingest large Distance Matrixes (1000+ nodes) in Python OR-Tools. Let's see if we can do better.

### Context
- [VRP Example on the OR-Tools Website](https://developers.google.com/optimization/routing/vrp#entire_program1)
- Using `model_params.max_callback_cache_size` (Thank you James Marca for showing me this)
- Python OR-Tools


In [1]:
import platform, ortools
print(f'Python{platform.python_version()}')
print(f'ortools={ortools.__version__}')

Python3.10.6
ortools=9.4.1874



### The example from the docs

```python
# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
```

This works fine for small problems, but takes 5+ seconds for a 3400x3400 distance matrix.

Key things to understand are:

- `IndexToNode` is a C++ function called from Python. 
- `distance_callback` is a Python function called from C++. 
- There's probably an overhead associated with each switch between C++ and Python.
- `distance_callback` is called n^2 times (I imagine) to precache the distance matrix. In this example, `IndexToNode` is therefore called 2n^2 times. 
- One call to our `distance_callback` example above switches between C++ and Python probably 6 times (3 roundtrips).

Therefore: 
1. You probably want to do as little work as possible inside `distance_callback`
2. You especially don't want to call `IndexToNode` inside `distance_callback`

### What should we do instead?

Let's test some approaches. 

We'll need to time how long everything takes

In [2]:
import time
from contextlib import contextmanager

@contextmanager
def timer(description):
    start = time.time()
    yield
    duration = time.time() - start
    print(f'{description}: {duration:.2f}s')

And we want to start with a fresh `RoutingModel` for each benchmark run.

In [3]:
from ortools_website_vrp_example import create_data_model
from ortools.constraint_solver import pywrapcp

data = create_data_model()

dist_mat = [row * 200 for row in data['distance_matrix']] * 200
assert len(dist_mat) == len(dist_mat[0]) == 3400

def setup():
    manager = pywrapcp.RoutingIndexManager(
        len(dist_mat),
        data['num_vehicles'], 
        data['depot'],
    )

    model_params = pywrapcp.DefaultRoutingModelParameters()
    model_params.max_callback_cache_size = 23_215_298
    routing = pywrapcp.RoutingModel(manager, model_params)
    return manager, routing


3400x3400 is a good size distance matrix to test performance on.

Let's try the example from the website

In [4]:
manager, routing = setup()
example = 'Website Example'

with timer(f'{example} Total Time'):

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return dist_mat[from_node][to_node]

    with timer(f'{example} Register Transits Time'):
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

Website Example Register Transits Time: 5.54s
Website Example Total Time: 5.54s


Pretty slow.

What if we do the IndexToNode lookup in advance, and store it in a dictionary?



In [5]:
manager, routing = setup()
example = 'Precalculate IndexToNode lookup'

with timer(f'{example} Total Time'):
    n_indices = manager.GetNumberOfIndices()
    lookup = {
        index: manager.IndexToNode(index)
        for index in range(n_indices)
    }
    
    def distance_callback(from_index, to_index):
        from_node = lookup[from_index]
        to_node = lookup[to_index]
        return dist_mat[from_node][to_node]

    with timer(f'{example} Register Transits Time'):
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

Precalculate IndexToNode lookup Register Transits Time: 1.77s
Precalculate IndexToNode lookup Total Time: 1.77s


3x faster already! Nice!!

But could we get rid of the `lookup[from_index]` and `lookup[to_index]`... ?

**Yes!**  
We could precalculate another representation of the distance matrix **knowing** that it will be indexed using `matrix[from_index][to_index]` not `matrix[from_node][to_node]`.

In [6]:
manager, routing = setup()
example = 'Precalculate Index-Based Distance Matrix'

with timer(f'{example} Total Time'):

    def create_idx_based_matrix(manager, dist_mat):
        n_indices = manager.GetNumberOfIndices()
        lookup = {
            index: manager.IndexToNode(index)
            for index in range(n_indices)
        }
        return [[
                dist_mat[lookup[from_index]][lookup[to_index]]
                for to_index in range(n_indices)
            ] for from_index in range(n_indices)
        ]

    idx_based_dist_mat = create_idx_based_matrix(manager, dist_mat)

    def distance_callback(from_index, to_index):
        # no need to convert indexes to nodes 
        # *in* the callback anymore
        return idx_based_dist_mat[from_index][to_index]

    with timer(f'{example} Register Transits Time'):
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

Precalculate Index-Based Distance Matrix Register Transits Time: 1.06s
Precalculate Index-Based Distance Matrix Total Time: 2.15s


Hmmmmm... We've reduced the time that `RegisterTransitCallback` takes, but increased the total time because we have to perform a lot of operations to make the Index-Based Distance Matrix in the first place.

Perhaps we can precalculate it more efficiently:

In [7]:
manager, routing = setup()
example = 'Better Precalculate Index-Based Distance Matrix'

with timer(f'{example} Total Time'):

    def create_idx_based_matrix(manager, dist_mat):
        n_indices = manager.GetNumberOfIndices()
        
        # Actually the lookup doesn't need to be a dict.
        # The keys were all just consecutive integers starting at 0...
        # And that's exactly how you index a tuple or list anyway.
        lookup = tuple(manager.IndexToNode(index) for index in range(n_indices))

        # ALSO, in this list comp, we're basically just iterating through
        # `lookup` item by item. Python has a better way to do that, 
        # and it's probably faster...
        return [[
                dist_mat[from_node][to_node]
                for to_node in lookup
            ] for from_node in lookup
        ]

    idx_based_dist_mat = create_idx_based_matrix(manager, dist_mat)

    def distance_callback(from_index, to_index):
        return idx_based_dist_mat[from_index][to_index]

    with timer(f'{example} Register Transits Time'):
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

Better Precalculate Index-Based Distance Matrix Register Transits Time: 1.18s
Better Precalculate Index-Based Distance Matrix Total Time: 1.69s


The total time is only slightly faster than the "Precalculate IndexToNode lookup" example. BUT, the time to register the callback is 60% faster.

So you don't get much benefit if you're only registering one callback. BUT, a lot of us:
- Reuse the same matrix to make callbacks for each Vehicle, or
- Reuse the same matrix for `SetArcCostEvaluatorOfVehicle`

so by pulling out this cost up front, we will reduce the total time before OR-Tools starts solving. Nice!

### Oh. And one more thing...

`RegisterTransitCallback` does this:
```c++
const int size = Size() + vehicles(); 
std::vector<int64> cache(size * size, 0); 
for (int i = 0; i < size; ++i) { 
    for (int j = 0; j < size; ++j) { 
    cache[i * size + j] = callback(i, j); 
    } 
} 
```

Wouldn't it be better if we could somehow skip the part where it loops through all those cross language callbacks?

### `RegisterTransitMatrix`
**Full credit to James Marca for telling me about [this undocumented gem](https://github.com/google/or-tools/blob/82750ac12f1ee5354e1c7869894d9af3508778f2/ortools/constraint_solver/routing.cc#L1276)**


Basically, if you have a pre-computed matrix, just give it to `RegisterTransitMatrix` (or equivalent `RegisterUnaryTransitVector` for unary callbacks), and it avoids all switching back and forth between Python and C++.

You don't even need to make it an index-base distance matrix..

In [8]:
manager, routing = setup()
example = 'RegisterTransitMatrix'

with timer(f'{example} Total Time'):
    with timer(f'{example} Register Transits Time'):
        transit_callback_index = routing.RegisterTransitMatrix(dist_mat)

RegisterTransitMatrix Register Transits Time: 0.16s
RegisterTransitMatrix Total Time: 0.16s


## So there's a pretty clear winner: `RegisterTransitMatrix`
As you can see:
- It's' **20x faster than the example on the OR-Tools website**
- It's 6x faster than giving the even-more-optimised index-based distance matrix to `RegisterTransitCallback`
- It is substantially less code.

### How does it work?
Looking at [`ortools/constraint_solver/routing.cc`](https://github.com/google/or-tools/blob/82750ac12f1ee5354e1c7869894d9af3508778f2/ortools/constraint_solver/routing.cc#L1276) I saw that `RegisterTransitMatrix` calls `RegisterTransitCallback` anyway!!! 

So how could it possibly be faster?
- It converts your matrix into a c++ matrix `std::vector<std::vector<int64_t>`
- It still has to call IndexToNode n^2 times, BUT now it's doing it in C++ on a C++ matrix.
- Your computer can probably do this calculation really fast since it doesn't have to switching between languages 6 times in each callback.

## What's the impact in the real world?
In our product, we have:
- a Transit Time callback for each vehicle
- a Total Time callback for each vehicle
- two other callbacks which we use for very specific business constraints.

I tested using an example with 1000 Jobs and 30 Vehicles.

|                                                 | Time to Register all callbacks   |  Code in our `callbacks.py`|
| -------------                                   | -------------           |  ----------   |
| Before this investication                       | 80 seconds              |  145 lines    |
| With `Precalculate Index-Based Distance Matrix` | 28 seconds              | 180 lines |
| With `RegisterTransitMatrix`                    | 5 seconds               |    100 lines  |


Many of our customers lose money for every minute they delay the drivers' departure. This is incredibly valuable.

## Numpy
Our software passes the distance matrix around as a `np.ndarray` most of the time, since it's better for slicing, broadcast multiplication, etc. 

Before learning about `RegisterTransitMatrix`, I benchmarked `RegisterTransitCallback` with the Distance Matrix as a numpy array.

### Should I give a `np.ndarray` to `RegisterTransitMatrix`?
You can't.
```python
routing.RegisterTransitMatrix(np.array(dist_mat))
# TypeError: Expecting a list of tuples
```


### Should I give a `np.ndarray` to `RegisterTransitCallback`?
Let's check.

In [9]:
manager, routing = setup()
example = 'Precalculate Index-Based Distance Matrix -> Numpy'


import numpy as np

with timer(f'{example} Total Time'):

    # the same `create_idx_based_matrix` function as before, 
    # but we make it into a numpy array
    idx_based_dist_mat = np.array(
        create_idx_based_matrix(manager, dist_mat))

    def distance_callback(from_index, to_index):
        return idx_based_dist_mat[from_index, to_index]

    with timer(f'{example} Register Transits Time'):
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

Precalculate Index-Based Distance Matrix -> Numpy Register Transits Time: 2.35s
Precalculate Index-Based Distance Matrix -> Numpy Total Time: 3.55s


To my surprise, the **RegisterTransitCallback is 2x slower with a numpy array** than with a Python list of lists.

For the record:
- I tried `np.array(..., order='C')` and `np.array(..., order='F')`. F is slower than C. C is as fast as the example above without `order=`, so I assume it was already C.
- I tried `idx_based_dist_mat[from_index][to_index]` as well. `idx_based_dist_mat[from_index, to_index]` is faster.
- There are faster ways to convert your `dist_mat` into a numpy `idx_based_dist_mat` (change the listcomp to a generator compt then use `np.fromiter`), but the bottleneck here is Registering the callback... so I would say don't bother.

I don't know Numpy well - if there's some trick I'm missing, please let me know!

### What should I do if my Distance Matrix is a `np.ndarray` to start?
Use `np.ndarray().tolist()` and `RegisterTransitMatrix`

In [10]:
manager, routing = setup()
example = 'Numpy -> RegisterTransitMatrix'

dist_mat_np = np.array(dist_mat)

with timer(f'{example} Total Time'):
    list_of_lists = dist_mat_np.tolist()
    with timer(f'{example} Register Transits Time'):
        transit_callback_index = routing.RegisterTransitMatrix(list_of_lists)

Numpy -> RegisterTransitMatrix Register Transits Time: 0.26s
Numpy -> RegisterTransitMatrix Total Time: 0.54s


## Other Gotchas

### Doing calculation in the callback
Any calculation you do inside the callback is slowing you down.
For example, in our codebase we had something like
```python
def callback(from_idx, to_idx):
    return dist_mat[from_idx][to_idx] / driver.speed_multiplier
```

This slows down ingestion time. Calculate the matrix before the callback, and give it to `RegisterTransitMatrix`

### What type to give to the callback
In the current version of OR-Tools, it gives an error if you give it anything other than Integers in your Distance Matrix.

We're using an older version of OR-Tools, and **I think** we were giving it a matrix of floats. In one of my early tests (which lead to deciding to do this benchmark), I realised that giving it integers instead of floats lead to a massive speedup.

# Results

**Register Duration**: The time it takes to call `RegisterTransitCallback` or `RegisterTransitMatrix`  
**Total Duration**: The time it takes to reformat the Distance Matrix for that approach, plus the **Register Duration**

| Approach                                          | Register Duration | Total Duration |
| -------------                                     | -------------      |  ----------   |
| Website Example                                   | 5.6s | 5.6s |
| Precalculate IndexToNode lookup                   | 1.9s | 1.9s |
| Precalculate Index-Based Distance Matrix          | 1.1s | 2.2s |
| Better Precalculate Index-Based Distance Matrix   | 1.1s | 1.6s |
| RegisterTransitMatrix                             | 0.2s | 0.2s |
| Precalculate Index-Based Distance Matrix -> Numpy | 2.2s | 3.2s |
| Numpy -> RegisterTransitMatrix                    | 0.2s | 0.5s |



## Conclusion
- Always use `RegisterTransitMatrix` with `max_callback_cache_size` if you use Python OR-Tools
- I'm surprised the documentation doesn't recommend this anyway.